In [15]:
import warnings

warnings.filterwarnings('ignore')

import os

from tools.modeling.timeseries.timeseriesmodeler import TimeSeriesModeler

from nba.common import add_game_log_lag, add_game_log_last_5
import nba.constants as c

from xgboost import XGBRegressor
from hyperopt import hp

import numpy as np
import pandas as pd
import pickle

ROOT_DIR = os.getcwd()[:-26]  # verify this points to the root directory of the project

season = c.SEASON_2022_2023

ROOT_DIR

'/Users/rmcdevi2/Projects/personal/nba-analysis/'

In [16]:
logs = pd.read_csv(ROOT_DIR + '/data/game_logs/{season}/top_100_scorers_{season}_game_log.csv'.format(season=season))
logs

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON
0,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201198,2023-04-06,PHI vs. MIA,L,...,2,2,0,2,0,1,4,21,-11,2022-23
1,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201181,2023-04-04,PHI vs. BOS,W,...,6,3,0,2,1,3,9,52,17,2022-23
2,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201174,2023-04-02,PHI @ MIL,L,...,5,1,0,0,1,4,4,28,-10,2022-23
3,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201150,2023-03-31,PHI vs. TOR,W,...,1,5,0,1,0,3,6,25,0,2022-23
4,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201139,2023-03-29,PHI vs. DAL,W,...,3,4,1,1,1,4,6,25,10,2022-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6576,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200072,2022-10-28,CLE @ BOS,W,...,2,0,0,0,0,2,2,12,25,2022-23
6577,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200056,2022-10-26,CLE vs. ORL,W,...,1,1,1,1,0,5,5,18,11,2022-23
6578,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200039,2022-10-23,CLE vs. WAS,W,...,3,0,0,2,1,2,4,15,-1,2022-23
6579,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200032,2022-10-22,CLE @ CHI,W,...,2,2,0,1,0,2,3,8,5,2022-23


In [17]:
logs[c.GAME_DATE] = pd.to_datetime(logs[c.GAME_DATE])
logs = add_game_log_last_5(logs)
logs = add_game_log_lag(logs)

logs.sort_values(by=[c.GAME_DATE])
logs

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,FG_PCT_LAG,FGM_LAG,FGA_LAG,FG3M_LAG,FG3A_LAG,FG3_PCT_LAG,FTM_LAG,FTA_LAG,FT_PCT_LAG,PLUS_MINUS_LAG
552,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200002,2022-10-18,LAL @ GSW,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200016,2022-10-20,LAL vs. LAC,L,...,0.462,12.0,26.0,3.0,10.0,0.300,4.0,4.0,1.000,-10.0
550,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200037,2022-10-23,LAL vs. POR,L,...,0.412,7.0,17.0,2.0,8.0,0.250,4.0,6.0,0.667,-1.0
549,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200064,2022-10-26,LAL @ DEN,L,...,0.545,12.0,22.0,2.0,9.0,0.222,5.0,7.0,0.714,2.0
548,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200076,2022-10-28,LAL @ MIN,L,...,0.381,8.0,21.0,2.0,8.0,0.250,1.0,2.0,0.500,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201149,2023-03-31,IND vs. OKC,W,...,0.524,11.0,21.0,1.0,5.0,0.200,6.0,9.0,0.667,-11.0
4799,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201173,2023-04-02,IND @ CLE,L,...,0.625,5.0,8.0,1.0,3.0,0.333,4.0,5.0,0.800,-9.0
4798,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201190,2023-04-05,IND vs. NYK,L,...,0.417,5.0,12.0,1.0,2.0,0.500,8.0,8.0,1.000,-7.0
4797,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201203,2023-04-07,IND vs. DET,L,...,0.353,6.0,17.0,1.0,3.0,0.333,4.0,6.0,0.667,-31.0


In [18]:
# Preprocess Data
PTS_PREDICTION_MASK = [c.PTS] + c.GAME_LOG_LAG_MASK + c.GAME_LOG_LAST_5_MASK
logs = logs[PTS_PREDICTION_MASK]
logs = logs.dropna()

cats = logs.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
    logs[col] = logs[col].astype('category')

logs.columns

Index(['PTS', 'MIN_LAG', 'PTS_LAG', 'REB_LAG', 'AST_LAG', 'STL_LAG', 'BLK_LAG',
       'TOV_LAG', 'FG_PCT_LAG', 'FGM_LAG', 'FGA_LAG', 'FG3M_LAG', 'FG3A_LAG',
       'FG3_PCT_LAG', 'FTM_LAG', 'FTA_LAG', 'FT_PCT_LAG', 'PLUS_MINUS_LAG',
       'MIN_LAST_5', 'PTS_LAST_5', 'REB_LAST_5', 'AST_LAST_5', 'STL_LAST_5',
       'BLK_LAST_5', 'TOV_LAST_5', 'FG_PCT_LAST_5', 'FGM_LAST_5', 'FGA_LAST_5',
       'FG3M_LAST_5', 'FG3A_LAST_5', 'FG3_PCT_LAST_5', 'FTM_LAST_5',
       'FTA_LAST_5', 'FT_PCT_LAST_5', 'PLUS_MINUS_LAST_5'],
      dtype='object')

In [5]:
rmse = {
    'base_model': 0,
    'tuned_model': 0,
}

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
}

In [6]:
# Base Model
modeler = TimeSeriesModeler(
    XGBRegressor(),
    logs,
    c.PTS,
)
modeler.train(params)

[0]	validation_0-rmse:8.01479	validation_1-rmse:9.43432
[53]	validation_0-rmse:1.14628	validation_1-rmse:9.64245
Cross Validation Iteration: 1
[0]	validation_0-rmse:8.61799	validation_1-rmse:8.13083
[53]	validation_0-rmse:2.58794	validation_1-rmse:8.48235
Cross Validation Iteration: 2
[0]	validation_0-rmse:8.44802	validation_1-rmse:8.77489
[55]	validation_0-rmse:3.27164	validation_1-rmse:8.69173
Cross Validation Iteration: 3
[0]	validation_0-rmse:8.48018	validation_1-rmse:8.43761
[53]	validation_0-rmse:3.96389	validation_1-rmse:8.53881
Cross Validation Iteration: 4
[0]	validation_0-rmse:8.46086	validation_1-rmse:7.59782
[55]	validation_0-rmse:4.32370	validation_1-rmse:7.49304
Cross Validation Iteration: 5


In [7]:
modeler.score

8.127687499410696


In [8]:
space = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'max_depth': hp.choice('max_depth', [3, 5, 7, 9]),
    'gamma': hp.uniform('gamma', 0, 9),
    'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.05, 0.5, 0.01),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'eta': hp.quniform('eta', 0.025, 1, 0.025),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.2),
}

In [9]:
best_features, best_params = modeler.hypertune_and_feature_selection(space)


250 trials detected for ('objective', 'eval_metric', 'n_estimators', 'early_stopping_rounds', 'max_depth', 'gamma', 'reg_alpha', 'reg_lambda', 'colsample_bytree', 'colsample_bylevel', 'min_child_weight', 'eta', 'subsample', 'learning_rate')
trial: 0001 ### iterations: 00047 ### eval_score: 8.79171
trial: 0002 ### iterations: 00074 ### eval_score: 8.88727
trial: 0003 ### iterations: 00070 ### eval_score: 8.85654
trial: 0004 ### iterations: 00242 ### eval_score: 8.89541
trial: 0005 ### iterations: 00039 ### eval_score: 8.90531
trial: 0006 ### iterations: 00021 ### eval_score: 8.82064
trial: 0007 ### iterations: 00027 ### eval_score: 8.92566
trial: 0008 ### iterations: 00174 ### eval_score: 8.85751
trial: 0009 ### iterations: 00273 ### eval_score: 8.8276
trial: 0010 ### iterations: 00367 ### eval_score: 8.82611
trial: 0011 ### iterations: 00340 ### eval_score: 8.80394
trial: 0012 ### iterations: 00188 ### eval_score: 8.83099
trial: 0013 ### iterations: 00039 ### eval_score: 8.88993
trial

In [10]:
with open('best_features.txt', 'wb') as f:
    pickle.dump(best_features, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('best_params.txt', 'wb') as f:
    pickle.dump(best_params, f)

In [11]:
mask = [c.PTS] + list(best_features)
features_selected_logs = logs[mask]
features_selected_logs

,PTS,FGA_LAG,PTS_LAST_5,FGM_LAST_5,FGA_LAST_5,FTM_LAST_5
547,26,24.0,25.8,9.8,22.0,4.2
546,20,22.0,24.8,9.4,21.2,4.2
545,17,23.0,24.8,9.8,22.4,3.8
544,27,19.0,22.0,8.8,21.8,3.4
543,30,23.0,23.6,9.8,22.2,3.2
...,...,...,...,...,...,...
4800,15,21.0,15.2,5.2,10.6,3.6
4799,19,8.0,15.2,5.6,11.2,3.2
4798,17,12.0,18.6,6.4,12.8,4.8
4797,16,17.0,21.2,7.2,15.4,5.6


In [12]:
new_modeler = TimeSeriesModeler(
    XGBRegressor(),
    features_selected_logs,
    c.PTS,
)
new_modeler.train(best_params)

[0]	validation_0-rmse:8.84565	validation_1-rmse:9.69603
[62]	validation_0-rmse:7.60693	validation_1-rmse:8.98870
Cross Validation Iteration: 1
[0]	validation_0-rmse:9.22182	validation_1-rmse:8.23898
[76]	validation_0-rmse:7.93858	validation_1-rmse:7.81588
Cross Validation Iteration: 2
[0]	validation_0-rmse:8.91440	validation_1-rmse:9.15907
[100]	validation_0-rmse:7.79753	validation_1-rmse:7.98154
[134]	validation_0-rmse:7.73027	validation_1-rmse:7.99884
Cross Validation Iteration: 3
[0]	validation_0-rmse:8.94755	validation_1-rmse:8.58130
[68]	validation_0-rmse:7.85520	validation_1-rmse:8.03092
Cross Validation Iteration: 4
[0]	validation_0-rmse:8.88073	validation_1-rmse:7.75651
[86]	validation_0-rmse:7.83597	validation_1-rmse:7.06187
Cross Validation Iteration: 5


In [13]:
new_modeler.score

7.924425490092451


In [14]:
# Load saved tuned params and features
with open('best_features.txt', 'rb') as f:
    best_features = pickle.load(f)

with open('best_params.txt', 'rb') as f:
    best_params = pickle.load(f)

print(best_features)
print(best_params)

Index(['FGA_LAG', 'PTS_LAST_5', 'FGM_LAST_5', 'FGA_LAST_5', 'FTM_LAST_5'], dtype='object')
{'colsample_bylevel': 0.17, 'colsample_bytree': 0.5067032703599714, 'early_stopping_rounds': 50, 'eta': 0.47500000000000003, 'eval_metric': 'rmse', 'gamma': 5.665965006013734, 'learning_rate': 0.19600644246709256, 'max_depth': 3, 'min_child_weight': 6.0, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'reg_alpha': 41.0, 'reg_lambda': 0.11456389545394619, 'subsample': 0.65}
